In [1]:
import luigi
import dill

from experiment_framework.helpers import AllAsymptotics, load_completed_AAs_into_dataframe
from empirical_privacy import one_bit_sum

In [13]:
n = 7
p = 0.5
ds = {
    'n_trials': n, 'prob_success': p, 'gen_distr_type': 'binom',
}
asys = {
    'gen_sample_kwargs'  : {'generate_in_batch': True},
    'fitter'             : 'knn',
    'fitter_kwargs'      : {'neighbor_method': 'gyorfi'},
    'n_docs'                : 30,
    'n_trials_per_training_set_size': 10,
    'n_max'              : 2**13,
    'validation_set_size': 2**10,
    'p'                  : 0.9,
    't'                  : 0.01
}

All = AllAsymptotics(
        gen_sample_path='empirical_privacy.one_bit_sum.GenSampleOneBitSum',
        dataset_settings=ds,
        asymptotic_settings=asys)
luigi.build([All], local_scheduler=True, workers=4, log_level='ERROR')

True

In [14]:
AAs = All.requires()
DF = load_completed_AAs_into_dataframe(AAs)
n_docs = DF.doc_ind.nunique()
print(n_docs)

30


In [15]:
DF = DF[DF.n_max==DF.n_max.max()]
pc = 0.5+0.5*one_bit_sum.sd(n, p)

In [16]:
pc

0.65625000000000011

In [17]:
DF[DF['upper_bound']>=pc].shape

(20, 15)

In [18]:
DF.shape

(30, 15)